# Experiments on Cross Lingual Transfer for Intent Detection
## Santichai Pornavalai
## 10.2.2021

This notebook is used to run the experiments for Intent Only cross-lingual experiments using XLM-R. The blocks for training and testing are meant to be run individually and correspond to the experiments listed in the paper

In [1]:
from preprocessing.util import *
import pickle
import sklearn
import torch
import numpy as np
from simpletransformers.classification import ClassificationModel

torch.manual_seed(136)
#from model import *

We begin by preprocessing the data

In [2]:
mapping = {}
with open('preprocessing/label_map.json','r') as f:
    mapping = json.load(f)
    mapping = {int(k):v for k,v in mapping.items()}
    
    
# preprocess training and test files to pandas df

# eng train
en_df, en_mapping = df_format(("data/en/train-en.tsv"),mapping)

# eng eval
en_df_eval, en_mapping = df_format("data/en/eval-en.tsv",mapping)

# eng test
en_df_test, en_mapping = df_format("data/en/test-en.tsv",mapping)

# es train
es_df, es_mapping = df_format("data/es/train-es.tsv",mapping)

# es eval
es_df_eval, es_mapping = df_format("data/es/eval-es.tsv",mapping)

# es test
es_df_test, es_mapping = df_format("data/es/test-es.tsv",mapping)


# th train
th_df, th_mapping = df_format("data/th/train-th_TH.tsv",mapping)

# th eval
th_df_eval, th_mapping = df_format("data/th/eval-th_TH.tsv",mapping)

# th test
th_df_test, th_mapping = df_format("data/th/test-th_TH.tsv",mapping)

mapping_list = list(mapping.values())

opening data/en/train-en.tsv
opening data/en/eval-en.tsv
opening data/en/test-en.tsv
opening data/es/train-es.tsv
opening data/es/eval-es.tsv
opening data/es/test-es.tsv
opening data/th/train-th_TH.tsv
opening data/th/eval-th_TH.tsv
opening data/th/test-th_TH.tsv


In [3]:

# drop some duplicate values 
# This is perhaps unnecessary. 
en_train = en_df.drop_duplicates("text")
en_eval = en_df_eval.drop_duplicates("text")
en_test = en_df_test.drop_duplicates("text")

es_train = es_df.drop_duplicates("text")
es_eval = es_df_eval.drop_duplicates("text")
es_test = es_df_test.drop_duplicates("text")

th_train = th_df.drop_duplicates("text")
th_eval = th_df_eval.drop_duplicates("text")
th_test = th_df_test.drop_duplicates("text")

Since we do not know if the other experiments were trained on a combination of train and eval, we err on the safe side ignore the eval file

In [4]:
# en_full_train = pd.concat([en_train,en_eval])
# es_full_train = pd.concat([es_train,es_eval])
# th_full_train = pd.concat([th_train, th_eval])
# quick hack to by-pass combining en eval 

en_full_train = en_train
es_full_train = es_train
th_full_train = th_train

In [5]:
# we no longer use these paths.
# should be removed
path2model = "prelim_models/"
path2model_en = "prelim_models/en/"
path2model_es = "prelim_models/es/"
path2model_th = "prelim_models/th/"
#path2model_x = "/home/santi/BA/final_models/x/"

In [6]:
# mix create data for cross-lingual training
en_th_full_train = pd.concat([en_full_train,th_full_train])
en_es_full_train = pd.concat([en_full_train,th_full_train])

In [7]:

def avg_sent_l(df):
    return sum([len(l.split()) for l in df["text"]])/len(df)

def lexical_diversity(df):
    lexes = set()
    for l in df["text"]:
        for w in l.split():
            lexes.add(w)
    return len(lexes), lexes


def analyze_wrong(wrong_predictions,model):
    wrongs = [(inp.text_a,inp.label) for inp in wrong_predictions]
    wrong_preds, vecs = model.predict([t for t,l in wrongs])

    dom_corr = 0
    weak_dom = 0
    rem_alarms = ["reminder","alarm"]
    results = []

    for (text, lab_true), lab_pred in zip(wrongs,wrong_preds):

        lab_pred = mapping[lab_pred]
        lab_true = mapping[lab_true]
        dom_pred = lab_pred.split("/")[0]
        dom_true = lab_true.split("/")[0]

        if dom_pred == dom_true:
            dom_corr += 1

        if (dom_pred in rem_alarms) and (dom_true in rem_alarms):
            weak_dom += 1    

        results.append((text,lab_pred, lab_true))

        #print(text,"\t" ,lab_pred,"\t", lab_true,"\t", dom_pred,"\t", dom_true)

    return results, dom_corr/len(wrongs)

In [8]:
# metrics used for evaluation
macro = lambda x,y:  sklearn.metrics.f1_score(x,y, average= 'macro')
micro = lambda x,y:  sklearn.metrics.f1_score(x,y, average= 'micro')
report = lambda x,y:  sklearn.metrics.classification_report(x,y,digits = 5,labels = list(range(0,12)), target_names = mapping_list)
report_dict = lambda x,y:  sklearn.metrics.classification_report(x,y,digits = 5,output_dict = True,labels = list(range(0,12)),target_names = mapping_list)
accuracy = lambda x,y:  sklearn.metrics.accuracy_score(x,y)


In [9]:
def custom_eval(df, model, ex_name = "experiment 1", verbose = True):
    results, predictions_vs, wrongs = model.eval_model(df, macro=macro, micro=micro,accuracy=accuracy, report=report, report_dict = report_dict)
    results["name"] = ex_name
    
    false_preds,dom_acc = analyze_wrong(wrongs,model)
    results["wrong_predictions"] = false_preds
    results["domain_of_wrongs"] = dom_acc
    results["domain_accuracy"] = results["accuracy"] + (1-results["accuracy"])*dom_acc
    
    if verbose:
        print("results for experiment: ",ex_name)

        print(results["report"])
        print("domain accuracy: ",results["domain_accuracy"])
        
    return results

Here we gather some statistics about the datasets

In [10]:
# 

print("average sentence length")
print("en",avg_sent_l(en_train))
print("es",avg_sent_l(es_train))
print("th",avg_sent_l(th_train))


print("unique tokens")
print("en",lexical_diversity(en_train)[0])
print("es",lexical_diversity(es_train)[0])
print("th",lexical_diversity(th_train)[0])

average sentence length
en 7.69126027754818
es 7.678731678133413
th 8.280373831775702
unique tokens
en 3983
es 1849
th 1138


We load up a pretrained XLM-R model with a Max Ent layer for classification. Arguments are left pretty vanilla except fp16 which is not relevant for the results. 

In [11]:
#change the hyper-parameters here. 

args={"fp16": True,
      'learning_rate':1e-5,
      'num_train_epochs': 5,
      'reprocess_input_data': True,
      'overwrite_output_dir': True,
      'save_steps':-1,
      "save_model_every_epoch":False,
     }

In [12]:
experiment_results = {}

In [13]:
# train english model
# full train = train + eval
args["output_dir"] = "models/intent_en_train"
model= ClassificationModel('xlmroberta','xlm-roberta-base', num_labels=12, args=args)
model.train_model(en_full_train)

# test eng
results = custom_eval(en_test, model, "train_en_test_en")
experiment_results[results["name"]] = results

# test es
results = custom_eval(es_test, model, "train_en_test_es")
experiment_results[results["name"]] = results

# test th
results = custom_eval(th_test, model, "train_en_test_th")
experiment_results[results["name"]] = results

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weig

/home/santi/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)



results for experiment:  train_en_test_en
                          precision    recall  f1-score   support

reminder/cancel_reminder    1.00000   0.98944   0.99469       284
            weather/find    0.99911   1.00000   0.99956      3386
      alarm/cancel_alarm    0.98210   0.98874   0.98541       444
 reminder/show_reminders    0.99537   0.99078   0.99307       217
      alarm/snooze_alarm    0.95556   0.96629   0.96089        89
alarm/time_left_on_alarm    0.94048   0.97531   0.95758        81
      alarm/modify_alarm    0.90476   0.93443   0.91935       122
     weather/checkSunset    1.00000   0.94595   0.97222        37
    weather/checkSunrise    1.00000   1.00000   1.00000        25
       alarm/show_alarms    0.99038   0.97170   0.98095       212
   reminder/set_reminder    0.99613   0.99922   0.99767      1287
         alarm/set_alarm    0.99023   0.98382   0.98701      1236

                accuracy                        0.99299      7420
               macro avg    0.9

/home/santi/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/santi/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)



results for experiment:  train_en_test_es
                          precision    recall  f1-score   support

reminder/cancel_reminder    0.99412   0.98830   0.99120       171
            weather/find    0.93217   0.99793   0.96393       964
      alarm/cancel_alarm    0.95939   0.87097   0.91304       217
 reminder/show_reminders    0.98425   0.94697   0.96525       132
      alarm/snooze_alarm    0.44186   0.59375   0.50667        32
alarm/time_left_on_alarm    1.00000   1.00000   1.00000        28
      alarm/modify_alarm    0.58929   0.97059   0.73333        34
     weather/checkSunset    0.00000   0.00000   0.00000         2
    weather/checkSunrise    0.00000   0.00000   0.00000         0
       alarm/show_alarms    0.95283   0.91818   0.93519       110
   reminder/set_reminder    0.98879   0.70111   0.82047       629
         alarm/set_alarm    0.81636   0.98879   0.89434       535

               micro avg    0.90960   0.90960   0.90960      2854
               macro avg    0.7

/home/santi/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)



results for experiment:  train_en_test_th
                          precision    recall  f1-score   support

reminder/cancel_reminder    0.90000   0.09574   0.17308        94
            weather/find    0.88278   1.00000   0.93774       610
      alarm/cancel_alarm    0.58371   0.94161   0.72067       137
 reminder/show_reminders    0.72222   0.62903   0.67241        62
      alarm/snooze_alarm    0.00000   0.00000   0.00000        20
alarm/time_left_on_alarm    1.00000   0.90000   0.94737        20
      alarm/modify_alarm    0.23404   0.84615   0.36667        13
     weather/checkSunset    0.00000   0.00000   0.00000         0
    weather/checkSunrise    0.00000   0.00000   0.00000         0
       alarm/show_alarms    0.86667   0.16883   0.28261        77
   reminder/set_reminder    0.99194   0.46415   0.63239       265
         alarm/set_alarm    0.64960   0.93050   0.76508       259

               micro avg    0.76622   0.76622   0.76622      1557
               macro avg    0.5

In [14]:
# reset model
# train on full spanish
args["output_dir"] = "models/intent_es_train"

model= ClassificationModel('xlmroberta','xlm-roberta-base', num_labels=12, args=args)
model.train_model(es_full_train)

# test eng
results = custom_eval(en_test, model, "train_es_test_en")
experiment_results[results["name"]] = results


# test es
results = custom_eval(es_test, model, "train_es_test_es")
experiment_results[results["name"]] = results


# test th
results = custom_eval(th_test, model, "train_es_test_th")
experiment_results[results["name"]] = results

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weig

/home/santi/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



results for experiment:  train_es_test_en
                          precision    recall  f1-score   support

reminder/cancel_reminder    1.00000   0.90845   0.95203       284
            weather/find    0.98115   0.99941   0.99020      3386
      alarm/cancel_alarm    0.89597   0.95045   0.92240       444
 reminder/show_reminders    1.00000   0.84793   0.91771       217
      alarm/snooze_alarm    0.27273   0.03371   0.06000        89
alarm/time_left_on_alarm    0.93671   0.91358   0.92500        81
      alarm/modify_alarm    0.88298   0.68033   0.76852       122
     weather/checkSunset    0.00000   0.00000   0.00000        37
    weather/checkSunrise    0.00000   0.00000   0.00000        25
       alarm/show_alarms    0.96135   0.93868   0.94988       212
   reminder/set_reminder    0.96203   0.98446   0.97312      1287
         alarm/set_alarm    0.89333   0.97573   0.93271      1236

                accuracy                        0.95418      7420
               macro avg    0.7


results for experiment:  train_es_test_es
                          precision    recall  f1-score   support

reminder/cancel_reminder    0.98276   1.00000   0.99130       171
            weather/find    0.99586   0.99793   0.99689       964
      alarm/cancel_alarm    0.97664   0.96313   0.96984       217
 reminder/show_reminders    0.99213   0.95455   0.97297       132
      alarm/snooze_alarm    0.96429   0.84375   0.90000        32
alarm/time_left_on_alarm    1.00000   1.00000   1.00000        28
      alarm/modify_alarm    1.00000   0.85294   0.92063        34
     weather/checkSunset    0.00000   0.00000   0.00000         2
    weather/checkSunrise    0.00000   0.00000   0.00000         0
       alarm/show_alarms    0.96296   0.94545   0.95413       110
   reminder/set_reminder    0.98736   0.99364   0.99049       629
         alarm/set_alarm    0.96527   0.98692   0.97597       535

               micro avg    0.98423   0.98423   0.98423      2854
               macro avg    0.8


results for experiment:  train_es_test_th
                          precision    recall  f1-score   support

reminder/cancel_reminder    0.00000   0.00000   0.00000        94
            weather/find    0.87231   0.99672   0.93037       610
      alarm/cancel_alarm    0.54545   0.91971   0.68478       137
 reminder/show_reminders    0.00000   0.00000   0.00000        62
      alarm/snooze_alarm    0.16667   0.05000   0.07692        20
alarm/time_left_on_alarm    1.00000   0.10000   0.18182        20
      alarm/modify_alarm    0.63636   0.53846   0.58333        13
     weather/checkSunset    0.00000   0.00000   0.00000         0
    weather/checkSunrise    0.00000   0.00000   0.00000         0
       alarm/show_alarms    0.62963   0.22078   0.32692        77
   reminder/set_reminder    0.89916   0.80755   0.85089       265
         alarm/set_alarm    0.74783   0.99614   0.85430       259

               micro avg    0.79191   0.79191   0.79191      1557
               macro avg    0.4

In [15]:
# reset model
args["output_dir"] = "models/intent_th_train"

model= ClassificationModel('xlmroberta','xlm-roberta-base', num_labels=12, args=args)
# train on full thai
model.train_model(th_full_train)

# test eng
results = custom_eval(en_test, model, "train_th_test_en")
experiment_results[results["name"]] = results


# test es
results = custom_eval(es_test, model, "train_th_test_es")
experiment_results[results["name"]] = results


# test th
results = custom_eval(th_test, model, "train_th_test_th")
experiment_results[results["name"]] = results

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weig


results for experiment:  train_th_test_en
                          precision    recall  f1-score   support

reminder/cancel_reminder    0.94340   0.17606   0.29674       284
            weather/find    0.87242   0.99970   0.93174      3386
      alarm/cancel_alarm    0.73542   0.79505   0.76407       444
 reminder/show_reminders    1.00000   0.08756   0.16102       217
      alarm/snooze_alarm    0.28571   0.02247   0.04167        89
alarm/time_left_on_alarm    0.93750   0.37037   0.53097        81
      alarm/modify_alarm    0.00000   0.00000   0.00000       122
     weather/checkSunset    0.00000   0.00000   0.00000        37
    weather/checkSunrise    0.00000   0.00000   0.00000        25
       alarm/show_alarms    0.18280   0.08019   0.11148       212
   reminder/set_reminder    0.78378   0.87879   0.82857      1287
         alarm/set_alarm    0.81246   0.92880   0.86674      1236

                accuracy                        0.82682      7420
               macro avg    0.5


results for experiment:  train_th_test_es
                          precision    recall  f1-score   support

reminder/cancel_reminder    0.86047   0.43275   0.57588       171
            weather/find    0.67226   0.99793   0.80334       964
      alarm/cancel_alarm    0.53929   0.69585   0.60765       217
 reminder/show_reminders    0.75000   0.06818   0.12500       132
      alarm/snooze_alarm    0.00000   0.00000   0.00000        32
alarm/time_left_on_alarm    1.00000   0.07143   0.13333        28
      alarm/modify_alarm    0.00000   0.00000   0.00000        34
     weather/checkSunset    0.00000   0.00000   0.00000         2
    weather/checkSunrise    0.00000   0.00000   0.00000         0
       alarm/show_alarms    0.01639   0.00909   0.01170       110
   reminder/set_reminder    0.82313   0.57711   0.67850       629
         alarm/set_alarm    0.81481   0.82243   0.81860       535

               micro avg    0.70147   0.70147   0.70147      2854
               macro avg    0.4


results for experiment:  train_th_test_th
                          precision    recall  f1-score   support

reminder/cancel_reminder    0.93878   0.97872   0.95833        94
            weather/find    0.99836   0.99836   0.99836       610
      alarm/cancel_alarm    0.89286   0.91241   0.90253       137
 reminder/show_reminders    0.95082   0.93548   0.94309        62
      alarm/snooze_alarm    1.00000   0.75000   0.85714        20
alarm/time_left_on_alarm    1.00000   0.95000   0.97436        20
      alarm/modify_alarm    0.00000   0.00000   0.00000        13
     weather/checkSunset    0.00000   0.00000   0.00000         0
    weather/checkSunrise    0.00000   0.00000   0.00000         0
       alarm/show_alarms    0.95946   0.92208   0.94040        77
   reminder/set_reminder    0.96691   0.99245   0.97952       265
         alarm/set_alarm    0.95149   0.98456   0.96774       259

               micro avg    0.96789   0.96789   0.96789      1557
               macro avg    0.7

In [16]:
args["output_dir"] = "models/intent_en_th_train"

model= ClassificationModel('xlmroberta','xlm-roberta-base', num_labels=12, args=args)
# train on full thai and eng mixed
model.train_model(en_th_full_train)
# test eng
results = custom_eval(en_test, model, "train_th_test_en")
experiment_results[results["name"]] = results


# test es
results = custom_eval(es_test, model, "train_th_test_es")
experiment_results[results["name"]] = results


# test th
results = custom_eval(th_test, model, "train_th_test_th")
experiment_results[results["name"]] = results

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weig


results for experiment:  train_th_test_en
                          precision    recall  f1-score   support

reminder/cancel_reminder    1.00000   0.99296   0.99647       284
            weather/find    1.00000   0.99970   0.99985      3386
      alarm/cancel_alarm    0.99099   0.99099   0.99099       444
 reminder/show_reminders    0.98624   0.99078   0.98851       217
      alarm/snooze_alarm    0.94565   0.97753   0.96133        89
alarm/time_left_on_alarm    0.94048   0.97531   0.95758        81
      alarm/modify_alarm    0.90551   0.94262   0.92369       122
     weather/checkSunset    1.00000   1.00000   1.00000        37
    weather/checkSunrise    1.00000   1.00000   1.00000        25
       alarm/show_alarms    0.98544   0.95755   0.97129       212
   reminder/set_reminder    0.99535   0.99845   0.99690      1287
         alarm/set_alarm    0.99024   0.98463   0.98742      1236

                accuracy                        0.99326      7420
               macro avg    0.9


results for experiment:  train_th_test_es
                          precision    recall  f1-score   support

reminder/cancel_reminder    0.98266   0.99415   0.98837       171
            weather/find    0.95804   0.99481   0.97608       964
      alarm/cancel_alarm    0.97549   0.91705   0.94537       217
 reminder/show_reminders    0.88276   0.96970   0.92419       132
      alarm/snooze_alarm    0.54545   0.37500   0.44444        32
alarm/time_left_on_alarm    0.96552   1.00000   0.98246        28
      alarm/modify_alarm    0.57627   1.00000   0.73118        34
     weather/checkSunset    0.00000   0.00000   0.00000         2
    weather/checkSunrise    0.00000   0.00000   0.00000         0
       alarm/show_alarms    0.95192   0.90000   0.92523       110
   reminder/set_reminder    0.99371   0.75358   0.85714       629
         alarm/set_alarm    0.82812   0.99065   0.90213       535

               micro avg    0.92256   0.92256   0.92256      2854
               macro avg    0.7


results for experiment:  train_th_test_th
                          precision    recall  f1-score   support

reminder/cancel_reminder    0.96842   0.97872   0.97354        94
            weather/find    0.99836   0.99672   0.99754       610
      alarm/cancel_alarm    0.97692   0.92701   0.95131       137
 reminder/show_reminders    0.93750   0.96774   0.95238        62
      alarm/snooze_alarm    1.00000   1.00000   1.00000        20
alarm/time_left_on_alarm    1.00000   0.95000   0.97436        20
      alarm/modify_alarm    0.91667   0.84615   0.88000        13
     weather/checkSunset    0.00000   0.00000   0.00000         0
    weather/checkSunrise    0.00000   0.00000   0.00000         0
       alarm/show_alarms    0.97368   0.96104   0.96732        77
   reminder/set_reminder    0.98127   0.98868   0.98496       265
         alarm/set_alarm    0.96226   0.98456   0.97328       259

               micro avg    0.98137   0.98137   0.98137      1557
               macro avg    0.8

In [17]:
args["output_dir"] = "models/intent_en_es_train"

model= ClassificationModel('xlmroberta','xlm-roberta-base', num_labels=12, args=args)
# train on full thai and eng mixed
model.train_model(en_es_full_train)
# test eng
results = custom_eval(en_test, model, "train_th_test_en")
experiment_results[results["name"]] = results


# test es
results = custom_eval(es_test, model, "train_th_test_es")
experiment_results[results["name"]] = results


# test th
results = custom_eval(th_test, model, "train_th_test_th")
experiment_results[results["name"]] = results

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weig


results for experiment:  train_th_test_en
                          precision    recall  f1-score   support

reminder/cancel_reminder    1.00000   0.99296   0.99647       284
            weather/find    0.99941   1.00000   0.99970      3386
      alarm/cancel_alarm    0.99099   0.99099   0.99099       444
 reminder/show_reminders    0.98165   0.98618   0.98391       217
      alarm/snooze_alarm    0.96703   0.98876   0.97778        89
alarm/time_left_on_alarm    0.95294   1.00000   0.97590        81
      alarm/modify_alarm    0.90551   0.94262   0.92369       122
     weather/checkSunset    1.00000   0.94595   0.97222        37
    weather/checkSunrise    1.00000   1.00000   1.00000        25
       alarm/show_alarms    1.00000   0.97170   0.98565       212
   reminder/set_reminder    0.99535   0.99767   0.99651      1287
         alarm/set_alarm    0.99024   0.98463   0.98742      1236

                accuracy                        0.99367      7420
               macro avg    0.9


results for experiment:  train_th_test_es
                          precision    recall  f1-score   support

reminder/cancel_reminder    0.96552   0.98246   0.97391       171
            weather/find    0.95054   0.99689   0.97316       964
      alarm/cancel_alarm    0.98413   0.85714   0.91626       217
 reminder/show_reminders    0.94776   0.96212   0.95489       132
      alarm/snooze_alarm    0.51515   0.53125   0.52308        32
alarm/time_left_on_alarm    0.93333   1.00000   0.96552        28
      alarm/modify_alarm    0.51515   1.00000   0.68000        34
     weather/checkSunset    0.00000   0.00000   0.00000         2
    weather/checkSunrise    0.00000   0.00000   0.00000         0
       alarm/show_alarms    0.92727   0.92727   0.92727       110
   reminder/set_reminder    0.99384   0.76948   0.86738       629
         alarm/set_alarm    0.85323   0.98879   0.91602       535

               micro avg    0.92362   0.92362   0.92362      2854
               macro avg    0.7


results for experiment:  train_th_test_th
                          precision    recall  f1-score   support

reminder/cancel_reminder    0.96842   0.97872   0.97354        94
            weather/find    0.99836   0.99672   0.99754       610
      alarm/cancel_alarm    0.98438   0.91971   0.95094       137
 reminder/show_reminders    0.95082   0.93548   0.94309        62
      alarm/snooze_alarm    0.95238   1.00000   0.97561        20
alarm/time_left_on_alarm    1.00000   1.00000   1.00000        20
      alarm/modify_alarm    0.91667   0.84615   0.88000        13
     weather/checkSunset    0.00000   0.00000   0.00000         0
    weather/checkSunrise    0.00000   0.00000   0.00000         0
       alarm/show_alarms    0.96154   0.97403   0.96774        77
   reminder/set_reminder    0.97407   0.99245   0.98318       265
         alarm/set_alarm    0.96958   0.98456   0.97701       259

               micro avg    0.98137   0.98137   0.98137      1557
               macro avg    0.8

Here are some sanity checks 

In [18]:
for text, predicted, real in experiment_results["train_en_test_th"]["wrong_predictions"]:
    print(text, "\t", predicted, "\t", real)

ฉัน ตั้ง นาฬิกา ปลุก อะไร ไว้ 	 alarm/set_alarm 	 alarm/show_alarms
ฉัน มี นาฬิกา ปลุก สําหรับ ตอน เช้า มั๊ย 	 alarm/set_alarm 	 alarm/show_alarms
ฉัน ตั้ง นาฬิกา ปลุก ไว้ กี่ ครั้ง 	 alarm/set_alarm 	 alarm/show_alarms
แสดง นาฬิกา ปลุก พรุ่ง นี้ ทั้งหมด 	 reminder/show_reminders 	 alarm/show_alarms
ฉัน มี เวลา อีก เท่า ไหร่ กว่า นาฬิกา ปลุก ดัง 	 alarm/set_alarm 	 alarm/time_left_on_alarm
ฉัน ได้ ตั้ง นาฬิกา ปลุก ไว้ สาม ทุ่ม หรือ ไม่ ? 	 alarm/set_alarm 	 alarm/show_alarms
ฉัน มี นาฬิกา ปลุก ตั้ง ไว้ หรือ ไม่ 	 alarm/set_alarm 	 alarm/show_alarms
นาฬิกา ปลุก ฉัน ตั้ง ไว้ กี่ โมง สําหรับ พรุ่ง นี้ 	 alarm/set_alarm 	 alarm/show_alarms
แสดง นาฬิกา ปลุก ของ ฉัน ทั้งหมด ให้ ฉัน ดู 	 reminder/show_reminders 	 alarm/show_alarms
เพิ่ม นาฬิกา ปลุก สําหรับ ปลุก เด็ก ทารก 	 alarm/modify_alarm 	 alarm/set_alarm
ตั้ง นาฬิกา ปลุก ทุก เช้า สัปดาห์ นี้ หรือ ไม่ ? 	 alarm/set_alarm 	 alarm/show_alarms
ตั้ง นาฬิกา ปลุก ตอน เช้า ตี 5 : 30 	 alarm/set_alarm 	 alarm/cancel_alarm
เมื่อ ไหร่ ฉัน ต้อง ตั้ง

เตือน ฉัน ให้ นํา ถัง ขยะ ออก ใน วัน พรุ่ง นี้ 	 alarm/set_alarm 	 reminder/set_reminder
เลื่อน การ เตือน ชั่วคราว 1 ชั่วโมง 	 alarm/modify_alarm 	 alarm/snooze_alarm
เลื่อน เวลา ปลุก 10นาที 	 alarm/modify_alarm 	 alarm/snooze_alarm
รี เซ็ ทนาฬิ กา ปลุก ทั้งหมด 	 weather/find 	 alarm/cancel_alarm
ตั้ง เวลา ปลุก อีก 20 นาที 	 alarm/modify_alarm 	 alarm/set_alarm
เอา เวลา ปลุก อาทิตย์ นี้ มา ดู หน่อย 	 weather/find 	 alarm/show_alarms
เลื่อน เวลา ปลุก ฉัน ไป 45 นาที 	 alarm/modify_alarm 	 alarm/snooze_alarm
แสดง สัญญาณ เตือน ภัย ของ ฉัน สําหรับ สัปดาห์ นี้ 	 weather/find 	 alarm/show_alarms
ปิด เสียง เตือน ชั่วคราว อีก 20 นาที 	 alarm/cancel_alarm 	 alarm/snooze_alarm
วัน พรุ่ง นี้ ฉัน จะ ตั้ง เวลา ปลุก อะไร 	 alarm/set_alarm 	 alarm/show_alarms
ตั้ง เวลา ปลุก ใหม่ 	 alarm/modify_alarm 	 alarm/cancel_alarm
กี่ โมง แล้ว ที่ เวลา ปลก ? 	 weather/find 	 alarm/show_alarms
เวลา ปลุก ตั้ง อยู่ ที่ อะไร ? 	 weather/checkSunset 	 alarm/show_alarms
ปลุก หลัง จาก 15 นาที 	 alarm/modify_alarm 	 ala

In [19]:
##### SANITY CHECK #####
def unique_sents(test_df, train_df):
    print("unique utterances in test data out of :", len(test_df))
    unique_sents = []
    train_set = set(train_df["text"])
    for sent in test_df["text"]:
        if sent not in train_set:
            unique_sents.append(sent)
    print(len(unique_sents)/len(test_df)*100,"% of the sentences are unique")

In [20]:
unique_sents(en_test,en_full_train)

unique utterances in test data out of : 7420
76.42857142857142 % of the sentences are unique


In [21]:
unique_sents(es_test, es_eval)
unique_sents(es_test,es_full_train)

unique utterances in test data out of : 2854
95.40995094604064 % of the sentences are unique
unique utterances in test data out of : 2854
92.67694463910301 % of the sentences are unique


In [22]:
unique_sents(th_test, th_eval)
unique_sents(th_test,th_full_train)

unique utterances in test data out of : 1557
93.7058445728966 % of the sentences are unique
unique utterances in test data out of : 1557
90.81567116249197 % of the sentences are unique


In [23]:
predict_sent = lambda sent: mapping[model.predict([sent])[0][0]]

In [24]:
predict_sent("what's the weather in Potsdam")

'weather/find'

In [25]:
predict_sent("don't wake me up tomorrow")

'alarm/cancel_alarm'

In [26]:

predict_sent("ตั้ง เวลา พรุ่ง บ่าย พรุ่งนี้")

'alarm/set_alarm'

In [27]:
predict_sent("que temperatura hay aqui")

'weather/find'

In [28]:
predict_sent("no necesito que levantarme el sabado" )

'alarm/set_alarm'

In [29]:
predict_sent("sabado no necesito que levantarme" )

'alarm/set_alarm'

In [30]:
predict_sent("ไม่ ต้อง ปลุก ฉัน วัน เสาร์ นะ" )

'alarm/cancel_alarm'

In [31]:
predict_sent("วัน เสาร์ ไม่ ต้อง ปลุก ฉัน นะ")

'alarm/cancel_alarm'

In [32]:
predict_sent("you don't have to wake me up on saturday")

'alarm/set_alarm'

In [33]:
predict_sent("saturday you don't have to wake me up")

'alarm/set_alarm'

In [34]:
en_full_train[en_full_train["text"].str.contains("^on (saturday|sunday|monday|tuesday)",case=False, regex=True)]

/home/santi/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,text,labels
21074,on saturday the 28th remind me about brewsters,10


In [35]:
mapping[10]

'reminder/set_reminder'

In [36]:
predict_sent("I don't have to wake up early on saturday")

'alarm/set_alarm'

In [37]:
# this is a weird sentence 
predict_sent("saturday you don't have to wake me up")

'alarm/set_alarm'

In [38]:
predict_sent("am Samstag musst du mich nicht aufwecken")

'alarm/set_alarm'

In [39]:
predict_sent("ich nicht muss aufstehen am Samstag")

'alarm/cancel_alarm'

In [40]:
"el sabado no necesito el despertador" 
# doesn't work
# implicit 

'el sabado no necesito el despertador'

In [41]:
"cuanto falta hasta el alarma"
"cuanto tiempo queda hasta que me levanto"
"que temperatura hay aqui"

'que temperatura hay aqui'